Here, you develop the 

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [15]:
nh = 8
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'U'
order = 2

This cell contains the relevant switch information.

In [16]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = ST.Upwind
else:
    DiffFunc = TT.CenterDiff#ST.CenterDiff

This cell instantiates the grid, derivative operators, and Fourier modes.

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, 'deriv')
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

In [5]:
polyInterp = GTT.GhostCellStencil(2, -0.5)
print(polyInterp)
print(type(polyInterp))

[ 0.5   0.75 -0.25]
<class 'numpy.ndarray'>


This cell runs a test on a fifth-order cell-averaged polynomial interpolation for a ghost cell at the edge of the coarse-fine interface. If it doesn't reach the appropriate order of accuracy, there is an `assert` which force quits.

In [14]:
TT.TestPoly(5, -0.5)

Order 0:
2.0 1.9999999999999996

Order 1:
-0.5 -0.5000000000000002

Order 2:
0.16666666666666666 0.1666666666666664

Order 3:
-0.0625 -0.06250000000000022

Order 4:
0.025 0.025000000000000577

Order 5:
-0.010416666666666666 -0.010416666666666297

Order 6:
0.004464285714285714 -1.6026785714285705



In [10]:
derivMat = OT.SpaceDeriv(omega, order, 'C')
print(derivMat)

[[-10.66666667  32.           0.           0.           0.           0.           0.           0.           0.           0.           0.         -21.33333333]
 [-32.           0.          32.           0.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.         -32.           0.          32.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.         -32.           0.          32.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.         -32.           0.          32.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.         -32.           0.          32.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0. 

In [11]:
omega2 = BT.Grid(nh)
finehalf2 = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega2.AddPatch(refRatio, finehalf2)

In [12]:
derivMat = OT.SpaceDeriv(omega2, order, 'C')
print(derivMat)

[[  0.          16.           0.           0.           0.           0.           0.           0.           0.           0.          -8.          -8.        ]
 [-16.           0.          16.           0.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.         -16.           0.          16.           0.           0.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.         -16.           0.           8.           8.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.         -21.33333333 -10.66666667  32.           0.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.         -32.           0.          32.           0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0. 